Step-by-Step Guide to Setting Up Federated Learning with TensorFlow Federated:
1. Install TensorFlow Federated

You first need to install the tensorflow-federated library.

In Google Colab, or your local Python environment, run:

In [ ]:
!pip uninstall -y tensorflow


In [ ]:
!pip install tensorflow==2.14.0


In [ ]:
import tensorflow as tf

# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")


TensorFlow Version: 2.14.0


In [ ]:
import tensorflow_federated as tff

# Check TensorFlow Federated version
print(f"TensorFlow Federated Version: {tff.__version__}")


TensorFlow Federated Version: 0.86.0


In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

# Step 1: Simulate client data
def create_client_data():
    """Simulate simple datasets for each client."""
    return [
        {
            "features": np.random.rand(10, 5),  # 10 samples, 5 features each
            "labels": np.random.randint(2, size=(10,))  # Binary classification (0 or 1)
        } for _ in range(3)
    ]

client_data = create_client_data()

# Step 2: Convert data to a TFF dataset
def create_tf_dataset_for_client(data):
    """Convert client data into TensorFlow datasets."""
    return tf.data.Dataset.from_tensor_slices((data["features"], data["labels"])).batch(2)

federated_train_data = [create_tf_dataset_for_client(client) for client in client_data]

# Step 3: Define a model using Keras
def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(5,)),  # 5 features
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Use tff.learning.models.Model to wrap the Keras model
    return tff.learning.models.from_keras_model(
        keras_model=model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Step 4: Build the federated learning process using the updated API
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

# Step 5: Initialize the federated learning process
state = iterative_process.initialize()

# Step 6: Run a few rounds of federated learning
for round_num in range(1, 11):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}, metrics={metrics}')


Round 1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.4), ('loss', 0.7357788), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.46666667), ('loss', 0.7263995), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.46666667), ('loss', 0.7179159), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Ro

Increase the Number of Rounds: You can train the model for more rounds to see further improvement in performance.

In [ ]:
for round_num in range(11, 21):  # Continue for 10 more rounds
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}, metrics={metrics}')


Round 11, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.53333336), ('loss', 0.6800981), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 12, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.53333336), ('loss', 0.6775046), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 13, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.56666666), ('loss', 0.67537344), ('num_examples', 30), ('num_batches', 15)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite',

Enhance the Model Architecture

In [ ]:
def enhanced_model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(5,)),  # 5 features
        tf.keras.layers.Dense(64, activation='relu'),  # Increased neurons
        tf.keras.layers.Dense(32, activation='relu'),  # Additional dense layer
        tf.keras.layers.Dense(16, activation='relu'),  # Additional dense layer
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    return tff.learning.models.from_keras_model(
        keras_model=model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )


2. Adjust the Learning Rate

 -  A higher learning rate can speed up convergence but might cause the model to miss optimal points.
 - A lower learning rate can slow down training but often leads to better convergence and performance.

Experimenting with different learning rates for both clients and the server can help improve accuracy. Here's an example where we adjust the learning rates:

In [ ]:
# Adjusted learning rates for clients and server
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=enhanced_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01),  # Lower learning rate
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.5)  # Adjusted server learning rate
)


3. Try a Different Optimizer

    - You can experiment with different optimizers like Adam, RMSprop, or Adagrad instead of SGD. These optimizers adapt their learning rates during training, which can improve convergence.

Example using Adam optimizer:

In [ ]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=enhanced_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.1)
)


4. Increase the Number of Training Rounds

Federated learning typically requires more rounds to achieve convergence, especially when each client has a limited dataset. You can run more training rounds to allow the model to learn better.